In [112]:
inbedpe = 'HICCUPS.bedpe.gz'
outbedpe = 'HICCUPS_loopMerged.bedpe'

In [113]:

bedpe = dict()

import gzip
with gzip.open(inbedpe, 'rb') as f:
    for line in f: 
        fields = line.rstrip().split("\t")   
        bedpe[fields[6]] = fields[0:6]
        bedpe[fields[6]].append('\n ')

In [114]:
import subprocess
subprocess.call("zcat " + inbedpe + " | cut -f 1,2,3,7 | bedtools sort | bedtools merge -d 10 -c 4 -o distinct > /tmp/startMerged.bed", shell = True)
subprocess.call("zcat " + inbedpe + " | cut -f 4,5,6,7 | bedtools sort | bedtools merge -d 10 -c 4 -o distinct > /tmp/stopMerged.bed", shell = True)

0

In [115]:
Alist = list()
Blist = list()

A = open('/tmp/startMerged.bed', 'r')
for line in A:
    fields = line.rstrip().split("\t")
    IDs = fields[3].split(',')
    IDs = map(int, IDs)
    IDs = set(IDs)
    Alist.append(IDs)
    
B = open('/tmp/stopMerged.bed', 'r')
for line in B:
    fields = line.rstrip().split("\t")
    IDs = fields[3].split(',')
    IDs = map(int, IDs)
    IDs = set(IDs)
    Blist.append(IDs)

In [116]:
Aset = map(set, Alist)
Bset = map(set, Blist)

In [117]:
MergableList = list()
for i in Aset:
    for j in Bset:
        O = i.intersection(j)
        if len(O) > 1:
            MergableList.append(O)

In [118]:
MergableList.sort(key=len, reverse=True)

In [119]:
OUT = open(outbedpe, 'w')
for e, SET in enumerate(MergableList):
    if e > 10: 
        exit
    lowestX1 = 0
    higestY2 = 0
    lowestX2 = 0
    higestY1 = 0
    c = 'a'
    for E, ID in enumerate(SET):
        ID = str(ID)
        if E == 0:
            c = bedpe[ID][0]
            lowestX1 = int(bedpe[ID][1])
            lowestY1 = int(bedpe[ID][4])
            higestX2 = int(bedpe[ID][2] )
            higestY2 = int(bedpe[ID][5] )
        else:
            if int(bedpe[ID][1]) < lowestX1:
                lowestX1 = int(bedpe[ID][1])
            if int(bedpe[ID][5]) > higestY2:
                higestY2 = int(bedpe[ID][5] )
            if int(bedpe[ID][1]) < lowestX2:
                lowestX2 = int(bedpe[ID][2])
            if int(bedpe[ID][4]) > higestY1:
                higestY1 = int(bedpe[ID][4] )
    OUT.write('\t'.join(     map(str,[c, lowestX1, higestX2,c,lowestY1,higestY2,'\n'])))

In [120]:
for ID in set(bedpe.keys()).symmetric_difference(set(map(str,[item for sublist in MergableList for item in sublist]))):
    OUT.write('\t'.join(bedpe[ID]))

In [121]:
OUT.close()